## 1. 데이터 전처리

해당 데이터를 전처리해서 허깅페이스에 데이터셋을 업로드하기까지의 과정은 아래의 Colab 주소에서 확인 가능합니다.  
https://colab.research.google.com/drive/1ZVwJ24AX92XnosIpS5-sbDqcKz_SloE9?usp=sharing

In [1]:
%pip install "torch==2.8.0"
%pip install "torchvision==0.23.0" 
%pip install "torchaudio==2.8.0"
%pip install "transformers==4.55.2"
%pip install "tokenizers==0.21.4"
%pip install "safetensors==0.6.2"
%pip install "huggingface-hub==0.34.4"
%pip install "datasets==4.0.0"
%pip install "accelerate==1.10.0"
%pip install "trl==0.21.0"
%pip install "peft==0.17.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 240.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 321.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 287.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 353.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 280.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 194.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 295.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 310.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 347.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 268.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 297.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2

In [2]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [3]:
from datasets import load_dataset

# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/finance_news_summarizer", split="train")

# 2. system_message 정의
# 데이터셋에 이미 포함된 system_prompt 열을 사용할 것이므로 따로 정의하지 않음

# 3. 원본 데이터의 type별 분포 출력
# 데이터셋에 type 열이 없으므로 전체 데이터 크기만 출력
print("전체 데이터 크기:", len(dataset))

# 4. train/test 분할 비율 설정 (0.5면 5:5로 분할)
test_ratio = 0.5

train_data = []
test_data = []

# 5. 전체 데이터의 인덱스를 train/test로 분할
data_indices = list(range(len(dataset)))
test_size = int(len(data_indices) * test_ratio)

test_data = data_indices[:test_size]
train_data = data_indices[test_size:]

# 6. OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": sample["system_prompt"],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            },
        ],
    }

# 7. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 8. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

README.md:   0%|          | 0.00/781 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/991 [00:00<?, ? examples/s]

전체 데이터 크기: 991

전체 데이터 분할 결과: Train 496개, Test 495개


In [4]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 

In [5]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [6]:
train_dataset[0]

{'messages': [{'content': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한

## 2. 모델 로드 및 템플릿 적용

In [7]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-4B" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [8]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들을 파이썬 문자열 리스트 형태로 작성

## 3. LoRA와 SFTConfig 설정

In [9]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. `0.1`로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. `"none"`으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. `"all"` 또는 `"lora_only"`와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 `"q_proj"`와 `"v_proj"`를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type`: LoRA가 적용되는 작업 유형을 지정합니다. `"CAUSAL_LM"`은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 `"SEQ2SEQ_LM"`(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [16]:
# 최대 길이
max_seq_length=16384

In [17]:
args = SFTConfig(
    output_dir="qwen3-4b-finance-new-summarizer",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=4,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None,
    max_length=max_seq_length,              # 최대 시퀀스 길이 추가
)

`output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜹니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [26]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 assistant 답변 부분을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [20]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 3200])
어텐션 마스크 형태: torch.Size([1, 3200])
레이블 형태: torch.Size([1, 3200])


In [21]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 55673, 31079, 85251, 5140, 231, 112, 24897, 138020, 98358, 87608, 19391, 126440, 129321, 17877, 55673, 16560, 5140, 231, 112, 24897, 134039, 140568, 126591, 42905, 40771, 230, 128024, 5140, 231, 112, 24897, 140568, 126591, 20487, 78952, 624, 126923, 127154, 143604, 3315, 120, 222, 12802, 24897, 19969, 134015, 130705, 125149, 136585, 54969, 12802, 144773, 20401, 10997, 141965, 76337, 42039, 140174, 16186, 139713, 624, 144544, 125686, 144707, 126414, 136541, 19391, 128772, 125686, 144707, 126414, 129125, 135968, 33509, 125476, 34395, 44518, 47985, 87425, 95577, 139713, 13, 23084, 16560, 10997, 54969, 144108, 17877, 60985, 98642, 128555, 128956, 129093, 135227, 72344, 48458, 13, 134084, 55673, 20401, 16186, 139713, 624, 52959, 53442, 10997, 56475, 126804, 897, 16560, 66790, 29326, 131193, 19391, 94613, 60838, 13, 66790, 55054, 131193, 17877, 126629, 135968, 21329, 125544, 139713, 13, 94613, 66790, 29326, 131193, 19391, 126629, 135

In [22]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [27]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):

<think>

</think>

{'is_stock_related': True, 'negative_impact_stocks': ['여신전문금융회사', '카드사'], 'negative_keywords': ['유동성 리스크', '리볼빙', '고위험 자산', '여신전문금융회사'], 'positive_impact_stocks': [], 'positive_keywords': [], 'reason_for_negative_impact': '금융감독원장이 유동성 리스크 관리를 강조하며 무리한 영업 자제와 리볼빙 관리 강화를 지시한 것은 여신전문금융회사와 카드사들에게 부정적인 영향을 미칠 수 있습니다. 특히, 고위험 자산 확대 및 무리한 영업 확장 자제가 요구되면서 수익성에 부정적인 영향을 줄 수 있습니다.', 'reason_for_positive_impact': '', 'summary': '금융감독원장이 카드사와 여신전문금융회사를 대상으로 무리한 영업 자제와 리볼빙 관리를 당부하며, 유동성 리스크와 취약차주 대출에 대한 주의를 강조했다. 이는 해당 금융사들의 수익성에 부정적인 영향을 미칠 수 있다.'}<|im_end|>


## input_ids와 labels는 어떻게 생성되는가?

Qwen3 모델의 챗 템플릿을 기준으로 input_ids와 labels 생성을 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.
사용자가 안녕하세요, 오늘 날씨는 어떤가요?라고 물었고,
모델은 안녕하세요! 오늘 날씨는 맑고 화창합니다.라고 응답해야 합니다.

Qwen3에서는 다음과 같은 템플릿 구조를 사용합니다:

```python
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>
<|im_start|>assistant
<think>

</think>

안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
```

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환됩니다. (해당 정수 시퀀스는 임의로 지정한 것으로 실제 정수와 다를 수 있습니다.)
```python
input_ids = [
    # <|im_start|>system\\nYou are a helpful assistant.<|im_end|>\\n
    1001, 1002, 13, 1003, 1004, 1005, 1006, 1007, 1008, 13,
    # <|im_start|>user\\n안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>\\n
    2001, 2002, 13, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 13,
    # <|im_start|>assistant\\n
    3001, 3002, 13,
    # <think>\\n\\n</think>\\n\\n
    4001, 13, 13, 4002, 13, 13,
    # 안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
    5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008
]
```
모델이 예측해야 할 영역은 assistant의 응답 부분인 <think></think> 태그와 최종 응답입니다.

따라서 labels는 다음과 같이 설정됩니다:
```python
labels = [
    # system 부분 마스킹
    -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
    # user 부분 마스킹
    -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
    # <|im_start|>assistant\\n 마스킹
    -100, -100, -100,
    # <think>\\n\\n</think>\\n\\n (마스킹 없음)
    4001, 13, 13, 4002, 13, 13,
    # 안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|> (마스킹 없음)
    5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008
]
```
이처럼 labels는 모델이 학습해야 할 assistant 응답 부분만을 포함하고, 나머지 부분은 -100으로 마스킹하여 손실 계산에서 제외됩니다. 모델은 `4001, 13, 13, 4002, 13, 13, 5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008`에 해당하는 `<think></think>안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>` 전체 응답 영역을 생성하도록 학습할 수 있습니다.

## 5. 학습

In [23]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [24]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,0.772100
20,0.551300
30,0.522400
40,0.527000
50,0.483200
60,0.502800
70,0.442900
80,0.437400
90,0.476700
100,0.464100


## 6. 테스트 데이터 준비

실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|im_start|>assistant'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [29]:
prompt_lst = []
label_lst = []
for prompt in test_dataset["messages"]:
   text = tokenizer.apply_chat_template(
       prompt, tokenize=False, add_generation_prompt=False
   )
   # <|im_start|>assistant\n<think>\n\n</think>\n\n까지 포함
   input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant\n<think>\n\n</think>\n\n'
   # <think>\n\n</think>\n\n 이후 부분만 label로
   label = text.split('<think>\n\n</think>\n\n')[1]
   prompt_lst.append(input)
   label_lst.append(label)

In [30]:
print(prompt_lst[10])

<|im_start|>system
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 되었던 주요한 명사 키워드들을 파이썬 문자열 리스트 형태로 작성

In [33]:
print(label_lst[10])

{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC', '텐센트', '바이트댄스'], 'negative_keywords': ['스마트폰 판매 감소', '인력 감축', '비용 절감', '경제 둔화'], 'positive_impact_stocks': [], 'positive_keywords': [], 'reason_for_negative_impact': '가트너의 보고서에 따르면 스마트폰 판매량의 감소는 애플과 같은 제조업체뿐만 아니라 엔비디아와 TSMC 같은 반도체 업체에게 부정적인 영향을 미칠 것으로 보입니다. 또한, 텐센트와 바이트댄스가 인력을 추가 감원할 계획을 발표함에 따라 이들의 주가에 부정적인 영향이 예상됩니다.', 'reason_for_positive_impact': '', 'summary': '가트너는 올해 전세계 스마트폰 판매량이 7% 감소할 것이라고 전망하며, 애플과 반도체 업체들이 영향을 받을 것으로 보입니다. 한편, 텐센트와 바이트댄스는 하반기 대규모 감원을 계획하고 있어, 글로벌 빅테크 기업들이 경제 둔화와 비용 절감 압박에 직면하고 있습니다.'}<|im_end|>



## 7. 파인 튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"qwen2-7b-rag-ko/checkpoint-285"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.

In [34]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [35]:
peft_model_id = "qwen3-4b-finance-new-summarizer/checkpoint-186"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [36]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [37]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

임의로 테스트 데이터 10번부터 14번까지 확인해봅시다.

In [38]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC'], 'negative_keywords': ['스마트폰 판매량 감소', '가트너', '애플', '엔비디아', 'TSMC'], 'positive_impact_stocks': [], 'positive_keywords': [],'reason_for_negative_impact': '스마트폰 판매량 감소 전망과 관련하여 애플, 엔비디아, TSMC 등 스마트폰 제조사와 반도체 업체들이 수요 부진으로 인한 압력이 가해질 것으로 예상되기 때문입니다.','reason_for_positive_impact': '','summary': '글로벌 비즈는 올해 전세계 스마트폰 판매량이 7% 감소할 것으로 전망하며, 중국의 봉쇄조치와 인플레이션으로 소비자들이 지갑을 열기 주저하고 있다는 점을 설명했습니다. 또한, EU가 가상자산을 이용한 돈세탁을 막기 위한 규제 움직임과 미국 저비용 항공사 스피릿 항공의 인수전, 중국 대표 기술기업들의 감원 움직임에 대해 다루었습니다.'}
    label:
{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC', '텐센트', '바이트댄스'], 'negative_keywords': ['스마트폰 판매 감소', '인력 감축', '비용 절감', '경제 둔화'], 'positive_impact_stocks': [], 'positive_keywords': [], 'reason_for_negative_impact': '가트너의 보고서에 따르면 스마트폰 판매량의 감소는 애플과 같은 제조업체뿐만 아니라 엔비디아와 TSMC 같은 반도체 업체에게 부정적인 영향을 미칠 것으로 보입니다. 또한, 텐센트와 바이트댄스가 인력을 추가 감원할 계획을 발표함에 따라 이들의 주가에 부정적인 영향이 예상됩니다.', 'reason_for_positive_

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['야놀자'], 'positive_keywords': ['야놀자', '포커스미디어', '동네가게 오래함께', '광고 제작', '소상공인', '지역사회'],'reason_for_negative_impact': '','reason_for_positive_impact': '야놀자는 포커스미디어와의 협력으로 지역 내 우수 소상공인을 발굴하고 맞춤형 광고를 제작하여 지역사회와 상생을 도모하는 캠페인을 진행함으로써 지역경제 활성화에 기여할 것으로 기대된다.','summary': '야놀자는 포커스미디어와 지역 내 우수 소상공인을 발굴하고 맞춤형 광고를 제작하여 지역사회와 상생을 도모하는 '동네가게 오래함께' 캠페인을 진행한다. 캠페인은 야놀자와 포커스미디어가 전액 부담하는 14억 원 규모의 광고 제작 및 송출을 통해 지역 내 소상공인의 인지도 제고와 매출 증대를 목표로 한다.'}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['야놀자'], 'positive_keywords': ['야놀자', '포커스미디어', '소상공인 지원', '광고 캠페인', '지역경제 활성화'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': '야놀자가 포커스미디어와 협력하여 지역 소상공인을 지원하는 캠페인을 진행함으로써 지역사회와의 상생 및 경제 활성화에 기여하며, 이는 브랜드 이미지와 매출 성장에 긍정적인 영향을 미칠 수 있다.', 'summary': "야놀자가 포커스미디어와 함께 지역 소상공인을 지원하기 위한 '동네가게 오래함께' 캠페인을 진행하여 소

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['삼성바이오로직스'], 'positive_keywords': ['삼성바이오로직스', 'MSD', '위탁생산계약', '2768억원'],'reason_for_negative_impact': '','reason_for_positive_impact': '삼성바이오로직스가 미국 제약기업 MSD와 2768억원 규모의 의약품 위탁생산 공급계약을 체결하여 매출 증대에 기여할 것으로 예상되기 때문입니다.','summary': '삼성바이오로직스가 미국 제약기업 MSD와 2768억원 규모의 의약품 위탁생산 공급계약을 체결하여 매출 증대에 기여할 것으로 예상됩니다.'}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['삼성바이오로직스'], 'positive_keywords': ['위탁생산계약', 'MSD', '매출 증대'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': '삼성바이오로직스가 미국 제약기업인 MSD와 2768억원 규모의 위탁생산계약을 체결하였으므로, 이는 회사의 매출 증대에 긍정적 영향을 미칠 수 있다.', 'summary': '삼성바이오로직스가 미국 제약기업 MSD와 2768억원 규모의 의약품 위탁생산 계약을 체결하였으며, 이는 회사의 매출 대비 상당한 규모로, 향후 매출 증가가 기대된다.'}<|im_end|>

--------------------------------------------------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['LG전자', 'SM엔터테인먼트'], 'positive_keywords': ['피트니스 캔디', '디지털 피트니스 콘텐츠', '스마트 가전', 'K POP', '디지털 라이프스타일'],'reason_for_negative_impact': '','reason_for_positive_impact': 'LG전자와 SM엔터테인먼트가 디지털 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 발표하며, 이는 디지털 기술과 K POP 콘텐츠를 결합한 새로운 시장 진출을 의미합니다. 이는 두 회사의 시장 점유율 확대와 수익 증가에 긍정적인 영향을 미칠 수 있습니다.','summary': 'LG전자와 SM엔터테인먼트가 디지털 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 발표하며, K POP과 디지털 기술을 결합한 새로운 시장 진출을 추진하고 있다.'}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['LG전자', 'SM엔터테인먼트'], 'positive_keywords': ['피트니스 캔디', '디지털 피트니스', 'LG전자', 'SM엔터테인먼트', 'K POP', '메타버스'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': "LG전자와 SM엔터테인먼트가 공동으로 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 출시하며 디지털 피트니스 시장에 진출하여, 양사의 디지털 기술력과 K POP 콘텐츠를 바탕으로 새로운 수익 창출과 시장 기회를 얻게 될 가능성이 높기 때문입니다.", 'summary': "LG전자와 SM엔터테인먼트는 디지털 피트니스 콘

## 8. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.

In [39]:
base_model_id = "Qwen/Qwen3-4B"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [40]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{"is_stock_related": True,
"positive_impact_stocks": [],
"reason_for_positive_impact": "",
"positive_keywords": [],
"negative_impact_stocks": ["애플", "엔비디아", "TSMC", "텐센트", "바이트댄스"],
"reason_for_negative_impact": "스마트폰 판매량 감소, 중국 봉쇄 조치, 인플레이션, 소비자 지갑 열기 주저, 수요 감소, 기술 업체 및 빅테크 기업의 압력 증가, 경제 침체 우려",
"negative_keywords": ["스마트폰 판매량", "봉쇄 조치", "인플레이션", "소비자 지갑", "수요 감소", "기술 업체", "빅테크 기업", "경제 침체"], 
"summary": "가트너는 올해 전세계 스마트폰 판매량이 7% 감소할 것으로 예측하며, 중국의 봉쇄 조치와 인플레이션으로 소비자 수요가 줄어들 것으로 분석했습니다. 또한, 애플, 엔비디아, TSMC, 텐센트, 바이트댄스 등 기술 업체와 빅테크 기업이 수요 감소로 인해 압력이 가중될 것으로 예상했습니다. EU는 가상자산을 이용한 돈세탁을 막기 위해 규제를 강화하고 있으며, 스피릿 항공의 인수전은 지연되고 있습니다."}
    label:
{'is_stock_related': True, 'negative_impact_stocks': ['애플', '엔비디아', 'TSMC', '텐센트', '바이트댄스'], 'negative_keywords': ['스마트폰 판매 감소', '인력 감축', '비용 절감', '경제 둔화'], 'positive_impact_stocks': [], 'positive_keywords': [], 'reason_for_negative_impact': '가트너의 보고서에 따르면 스마트폰 판매량의 감소는 애플과 같은 제조업체뿐만 아니라 엔비디아와 TSMC 같은 반도체 업체에게 부정적인 영향을 미칠 것으로 보입니다. 또한, 텐

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{"is_stock_related": True,
"positive_impact_stocks": ["야놀자", "포커스미디어"],
"reason_for_positive_impact": "이 뉴스는 야놀자와 포커스미디어가 지역 소상공인을 지원하는 캠페인을 진행함으로써 두 회사의 브랜드 인지도와 지역 경제 활성화에 긍정적인 영향을 줄 것으로 예상됩니다. 또한, 광고 제작 및 송출 비용을 전액 부담하는 점도 회사의 사회적 책임과 마케팅 역량을 강조하는 데 기여할 수 있습니다.",
"positive_keywords": ["야놀자", "포커스미디어", "지역 소상공인", "광고 제작", "광고 송출", "지역 경제 활성화"],
"negative_impact_stocks": [],
"reason_for_negative_impact": "",
"negative_keywords": [],
"summary": "야놀자는 포커스미디어와 '동네가게 오래함께' 캠페인을 진행한다고 발표했다. 이번 캠페인은 지역 내 우수 소상공인을 발굴하고 맞춤형 광고를 제작해 지역 홍보를 지원하며, 총 14억 원 규모의 광고 제작 및 송출 비용은 양사가 전액 부담한다. 첫 광고는 서울시 노원구, 동작구를 시작으로 2개월 간 방영되며, 연말까지 대상 범위를 지속 확대할 예정이다."}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['야놀자'], 'positive_keywords': ['야놀자', '포커스미디어', '소상공인 지원', '광고 캠페인', '지역경제 활성화'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': '야놀자가 포커스미디어와 협력하여 지역 소상공인을 지원하는 캠페인을 진행함으로써 지역사회와의 상생 및 경제 활성화에 기여하며, 이는

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{"is_stock_related": True,
"positive_impact_stocks": ["삼성바이오로직스"],
"reason_for_positive_impact": "삼성바이오로직스가 미국 제약기업 MSD와 2768억원 규모의 의약품 위탁생산 공급계약을 체결함으로써 기업의 매출과 수익성에 긍정적인 영향을 줄 것으로 예상된다.",
"positive_keywords": ["삼성바이오로직스", "MSD", "위탁생산계약", "의약품", "2768억원"],
"negative_impact_stocks": [],
"reason_for_negative_impact": "",
"negative_keywords": [],
"summary": "삼성바이오로직스는 미국 제약기업 MSD와 2768억2938만원 규모의 의약품 위탁생산 공급계약을 체결했다. 계약 기간은 2022년 7월 1일부터 2028년 12월 31일이며, 고객사의 수요 증가에 따라 계약금액이 3억8186만 달러로 증가할 수 있다."}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['삼성바이오로직스'], 'positive_keywords': ['위탁생산계약', 'MSD', '매출 증대'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': '삼성바이오로직스가 미국 제약기업인 MSD와 2768억원 규모의 위탁생산계약을 체결하였으므로, 이는 회사의 매출 증대에 긍정적 영향을 미칠 수 있다.', 'summary': '삼성바이오로직스가 미국 제약기업 MSD와 2768억원 규모의 의약품 위탁생산 계약을 체결하였으며, 이는 회사의 매출 대비 상당한 규모로, 향후 매출 증가가 기대된다.'}<|im_end|>

---------------------------------

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
{"is_stock_related": True,
"positive_impact_stocks": ["LG전자", "SM엔터테인먼트"],
"reason_for_positive_impact": "LG전자와 SM엔터테인먼트가 디지털 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 공동으로 출시함으로써 두 회사의 디지털 기술력과 콘텐츠 경쟁력이 강화될 것으로 예상됩니다. 이는 두 회사의 시장 점유율 확대와 관련된 긍정적인 영향을 줄 것으로 보입니다.",
"positive_keywords": ["LG전자", "SM엔터테인먼트", "피트니스 캔디", "디지털 피트니스 콘텐츠", "디지털 기술", "콘텐츠 경쟁력"],
"negative_impact_stocks": [],
"reason_for_negative_impact": "",
"negative_keywords": [],
"summary": "LG전자와 SM엔터테인먼트가 디지털 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 공동으로 발표했습니다. 이는 두 회사의 디지털 기술력과 콘텐츠 경쟁력을 강화하고, 피트니스 및 헬스케어 산업에 기여할 것으로 기대됩니다."}
    label:
{'is_stock_related': True, 'negative_impact_stocks': [], 'negative_keywords': [], 'positive_impact_stocks': ['LG전자', 'SM엔터테인먼트'], 'positive_keywords': ['피트니스 캔디', '디지털 피트니스', 'LG전자', 'SM엔터테인먼트', 'K POP', '메타버스'], 'reason_for_negative_impact': '', 'reason_for_positive_impact': "LG전자와 SM엔터테인먼트가 공동으로 피트니스 콘텐츠 브랜드 '피트니스 캔디'를 출시하며 디지털 피트니스 시장에 진출하여, 양사의 디지털 기술력과 K POP 콘텐츠를 바탕으로 새로운 수익 창출과 시장 기회를 얻게 될 가능성이 높기 때문입니다